In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pdb
from tqdm import tqdm

# LQR

In [9]:
# LQR formula
# Use torch tensor to calculate the LQR
def LQR_PF(A, B, N, Q, R, Qf, T, x0):
    """
    Input:
        A, B: system dynamics
        Qf: terminal cost
        Q, R: state and control cost
        N: cross term
        T: time horizon
        x0: initial state
    Return:
        P: Riccati matrix
        F: feedback gain
        J: optimal cost
    """

    P = list(range(T+1))
    F = list(range(T))
    P[T] = Qf
    for t in range(T-1, -1, -1):
        F[t] = -torch.inverse(R + B.t() @ P[t+1] @ B) @ (B.t() @ P[t+1] @ A + N.t())
        P[t] = Q + A.t() @ P[t+1] @ A - (A.t() @ P[t+1] @ B + N) @ torch.inverse(R + B.t() @ P[t+1] @ B) @ (B.t() @ P[t+1] @ A + N.t())

    J = 0
    x = x0
    for t in range(T):
        u = F[t] @ x
        if t == T-1:
            J += x.t() @ Qf @ x
        else:
            J += x.t() @ Q @ x + u.t() @ R @ u + x.t() @ N @ u
        x = A @ x + B @ u
    
    return P, F, J


In [10]:
# Test with 1D system
# f(x, u) = x + u
# J = 1.618 * x_final^2 + \sum x^2 + u^2
# A = 1, B = 1
# Q = 1, R = 1, Qf = 1.618, N = 0
# T = 5
# x0 = 1
# Verify the result with the formula. Correct.

A = torch.tensor([[1.]])
B = torch.tensor([[1.]])
Q = torch.tensor([[1.]])
R = torch.tensor([[1.]])
Qf = torch.tensor([[1.618]])
N = torch.tensor([[0.]])
T = 5
x0 = torch.tensor([[1.]])
P, F, J = LQR_PF(A, B, N, Q, R, Qf, T, x0)

print("P:")
print(P)
print("F:")
print(F)
print("J:")
print(J)

P:
[tensor([[1.6180]]), tensor([[1.6180]]), tensor([[1.6180]]), tensor([[1.6180]]), tensor([[1.6180]]), tensor([[1.6180]])]
F:
[tensor([[-0.6180]]), tensor([[-0.6180]]), tensor([[-0.6180]]), tensor([[-0.6180]]), tensor([[-0.6180]])]
J:
tensor([[1.6180]])


# MPC_simple

In [11]:
# Construct MPC problem
# Use float32

DTYPES = torch.float32
torch.set_default_dtype(DTYPES)
MPC_T = 10
A = torch.tensor([[1.]])
B = torch.tensor([[1.]])
Q = torch.tensor([[1.]])
R = torch.tensor([[1.]])
Qf = nn.Parameter(torch.tensor([[1.618]]))
N = torch.tensor([[0.]])
T = 5
nx = 1
nu = 1

x_init = torch.tensor([[1.]])
x = x_init

x_list = []
u_list = []
VN_list = []
for i in range(MPC_T):
    P, F, J = LQR_PF(A, B, N, Q, R, Qf, T, x)
    u = F[0] @ x
    x_list.append(x)
    u_list.append(u)
    x = A @ x + B @ u
    VN_list.append(J)

print('x_list:')
print(x_list)
print('u_list:')
print(u_list)
print('VN_list:')
print(VN_list)

x_list:
[tensor([[1.]]), tensor([[0.3820]], grad_fn=<AddBackward0>), tensor([[0.1459]], grad_fn=<AddBackward0>), tensor([[0.0557]], grad_fn=<AddBackward0>), tensor([[0.0213]], grad_fn=<AddBackward0>), tensor([[0.0081]], grad_fn=<AddBackward0>), tensor([[0.0031]], grad_fn=<AddBackward0>), tensor([[0.0012]], grad_fn=<AddBackward0>), tensor([[0.0005]], grad_fn=<AddBackward0>), tensor([[0.0002]], grad_fn=<AddBackward0>)]
u_list:
[tensor([[-0.6180]], grad_fn=<MmBackward0>), tensor([[-0.2361]], grad_fn=<MmBackward0>), tensor([[-0.0902]], grad_fn=<MmBackward0>), tensor([[-0.0344]], grad_fn=<MmBackward0>), tensor([[-0.0132]], grad_fn=<MmBackward0>), tensor([[-0.0050]], grad_fn=<MmBackward0>), tensor([[-0.0019]], grad_fn=<MmBackward0>), tensor([[-0.0007]], grad_fn=<MmBackward0>), tensor([[-0.0003]], grad_fn=<MmBackward0>), tensor([[-0.0001]], grad_fn=<MmBackward0>)]
VN_list:
[tensor([[1.6180]], grad_fn=<AddBackward0>), tensor([[0.2361]], grad_fn=<AddBackward0>), tensor([[0.0344]], grad_fn=<AddB

## RDP

In [12]:
# Train the cost function use RDP method

def cost(x, u, is_terminal):
    if is_terminal:
        return x.t() @ Qf @ x
    else:
        return x.t() @ Q @ x + u.t() @ R @ u

def cal_RDP_criteria(VN_list, x_list, u_list, alpha, MPC_T, func):
    """
    Input:
        VN_list: list of the cost function value
        x_list: list of the state
        u_list: list of the control
        cost_nn: the cost function
        alpha: the weight of the RDP criteria
    Return:
        RDP criteria
    """
    loss = 0
    for i in range(MPC_T - 1):
        RDP = (VN_list[i+1] + alpha * cost(x_list[i], u_list[i], False)) - VN_list[i] # Wish RDP <= 0
        loss += func(RDP)
    return loss

loss = cal_RDP_criteria(VN_list, x_list, u_list, 1, MPC_T, lambda x: torch.tanh(1000000 * x))
print(loss)
loss.backward()
print(Qf.grad)

tensor([[6.5484e-05]], grad_fn=<AddBackward0>)
tensor([[-453.1076]])


# Large Scale test

In [14]:
# Construct MPC problem
# Use float32

DTYPES = torch.float32
torch.set_default_dtype(DTYPES)
MPC_T = 10
A = torch.tensor([[1.]])
B = torch.tensor([[1.]])
Q = torch.tensor([[1.]])
R = torch.tensor([[1.]])
Qf = nn.Parameter(torch.tensor([[1.630]]))
N = torch.tensor([[0.]])
T = 5
nx = 1
nu = 1

n_epoch = 100

loss = 0
for epoch in tqdm(range(n_epoch)):
    x_init = torch.randn(1, 1) * 10
    x = x_init

    x_list = []
    u_list = []
    VN_list = []
    for i in range(MPC_T):
        P, F, J = LQR_PF(A, B, N, Q, R, Qf, T, x)
        u = F[0] @ x
        x_list.append(x)
        u_list.append(u)
        x = A @ x + B @ u
        VN_list.append(J)
    
    loss += cal_RDP_criteria(VN_list, x_list, u_list, 1, MPC_T, lambda x: torch.tanh(1000 * x))

loss.backward()
print(Qf.grad)

100%|██████████| 100/100 [00:01<00:00, 52.31it/s]


tensor([[-1792.8451]])
